In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import pandas as pd


bins = 32
with open ('calibration.npy', 'rb') as f:
    hmatrix = np.load(f)
    left_g_int = np.load(f)
    left_r_int = np.load(f)
    right_g_int = np.load(f)
    right_r_int =np.load(f)

with open('frame.npy', 'rb') as f:
    test_frame = np.load(f)

with open('segmented.npy', 'rb') as f:
    segmented = np.load(f)

print(hmatrix.shape,left_g_int.shape,left_r_int.shape,right_g_int.shape,right_r_int.shape,test_frame.shape, segmented.shape)
hmatrix1d = hmatrix.flatten()

In [ ]:
res = np.where(hmatrix == 490)
print(res)

In [ ]:
def image_segmentation(frame):
    # RG chromaticity of frame
    np.seterr(invalid='ignore')
    I = frame.sum(axis=2)
    I[I == 0] = 100000

    frame_r = frame[:,:,2] / I
    frame_g = frame[:,:,1] / I
    
    frame_r_int = (frame_r*(bins-1)).astype(int)
    frame_g_int = (frame_g*(bins-1)).astype(int)

    back_projection = np.zeros(frame_r.shape, dtype = 'uint8')
    back_projection = hmatrix1d[frame_g_int.flatten()*bins + frame_r_int.flatten()].reshape(frame_r.shape)
    
    #masked = cv2.bitwise_and(frame, frame, mask = back_projection)

    return hmatrix, back_projection

In [61]:
def blob_detection(frame):
    #plt.imshow(frame)
    #plt.show()
    _, thresh = cv2.threshold(frame, 30, 255, cv2.THRESH_BINARY)
    # Define a circular structuring element for dilation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    # Dilate the circles using the circular structuring element
    dilated = cv2.dilate(thresh, kernel)
    circles = cv2.HoughCircles(dilated,cv2.HOUGH_GRADIENT,1,20,param1=50,param2=30,minRadius=0,maxRadius=0)
    #circles = cv2.HoughCircles(dilated, cv2.HOUGH_GRADIENT, 1, minDist=30, maxRadius=-1)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        # Draw the detected circles on the original image
        for circle in circles[0,:]:
            x, y, r = circle
        # draw the center of the circle
            cv2.circle(dilated,(x,y),2,(0,255,0),3)
    return circles, dilated

In [ ]:
frame_width = 640
frame_height = 480
hmatrix, back_proj = image_segmentation(test_frame)
#conv = back_proj.astype(np.uint8)
plt.imshow(back_proj)
plt.show()


In [62]:
circles, thresh = blob_detection(back_proj.astype(np.uint8))

# Show the results
print(circles.shape)
plt.imshow(thresh)
plt.show()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:

# Draw the detected circles on the original image
for circle in circles[0,:]:
    x, y, r = circle
    cv2.circle(back_proj, (x, y), r, (0, 255, 0), 2)

plt.imshow(back_proj)
plt.show()